# Hypercube on a string

A demo of the famous tesseract on a string! More info can be found here: https://enki.ws/ganja.js/examples/pga_dyn.html

In [1]:
%pip install -q kingdon anywidget==0.9.9 ipywidgets==8.1.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from kingdon import Algebra
from sympy import cos, sin, Symbol
import ipywidgets

In [3]:
def euler(f,y,h):
    return [yi + h*fi for yi, fi in zip(y, f(*y))]

def RK4(f,y,h):
    k1 = f(*y)
    k2 = f(*[yi + 0.5*h*k1i for yi, k1i in zip(y, k1)])
    k3 = f(*[yi + 0.5*h*k2i for yi, k2i in zip(y, k2)])
    k4 = f(*[yi + h*k3i for yi, k3i in zip(y, k3)])
    return [yi + (h/3)*(k2i + k3i + (k1i + k4i)*0.5) 
            for yi, k1i, k2i, k3i, k4i in zip(y, k1, k2, k3, k4)]

Hint: try increasing $d$!

In [4]:
d = 2
alg = Algebra(d, 0, 1)

In [5]:
points = [
    alg.vector([1.0, *(float(x)-0.5 for x in bin(i)[2:].zfill(d))]).dual()
    for i in range(2**d)
]
edges = [
    [a, b]
    for i, a in enumerate(points)
    for j, b in enumerate(points)
    if not (i<=j or (i ^ j) & ((i ^ j) - 1))
]

In [6]:
# Gravitational force and spring constant
fg = alg.multivector(e02=-9.81)
spring_constant = 12
# attach = alg.evenmv(e=1, e02=-0.25) >> points[2**d-1]
attach = alg.evenmv(e=1, e02=-1) * points[2**d-1]

def force(M, B):
    gravitational = (M.reverse() >> fg).dual()
    hooke = -spring_constant*((M.reverse() >> attach) & points[2**d-1])
    damping = (-0.25 * B).dual()
    return hooke + gravitational + damping

state = [
    alg.evenmv(e=1, e01=0.0, e02=0.0, e12=0.0),           # The starting position and orientation in space. (identity) 
    alg.multivector(e01=0.0, e02=0.0, e12=-0.2)           # The starting linear and rotational velocity in body.
]

dState = lambda M, B: [                                   # Change in M and B
    -0.5 * M * B,
    (force(M, B) +  B.cp(B.dual())).undual()
]

In [7]:
def graph_func():
    global state
    
    for _ in range(10):
        state = euler(dState, state, 1/600)
    # state = RK4(dState, state, 1/60)
    new_edges = [[state[0] >> p for p in edge] for edge in edges]

    return [
        *new_edges,
        0x007799,
        attach, 
        [attach, state[0] >> points[2**d-1]],
    ]

alg.graph(
    graph_func,
    animate=True,
    scale=0.5,
    lineWidth=6,
)

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0', 'e01', 'e02', '0', '0', …